In [1]:
# (1) パッケージをインストールする（初回のみ）
# !pip install pandas scipy numpy

In [2]:
# (2) ライブラリの読み出し
import pandas as pd
import numpy as np
from scipy import stats
import random

In [3]:
# (3) データの読み込み
email_data = pd.read_csv("http://www.minethatdata.com/Kevin_Hillstrom_MineThatData_E-MailAnalytics_DataMiningChallenge_2008.03.20.csv")

In [4]:
# (4) データの準備
## 女性向けメールが配信されたデータを削除したデータを作成
male_df = email_data.query('segment != "Womens E-Mail"').reset_index(drop=True)
male_df['treatment'] = (male_df['segment'] == "Mens E-Mail").astype(int)

In [5]:
# (5) 集計による比較
## group_byとsummairseを使って集計
male_df.groupby('treatment')[['conversion', 'spend']].agg(['mean', 'count'])

conversion            spend       
                mean  count      mean  count
treatment                                   
0           0.005726  21306  0.652789  21306
1           0.012531  21307  1.422617  21307

In [6]:
# (6) t検定を行う
## (a)男性向けメールが配信されたグループの購買データを得る
mens_mail = male_df.query('treatment == 1')['spend']
## (b)メールが配信されなかったグループの購買データを得る
no_mail = male_df.query('treatment == 0')['spend']

In [7]:
## (a)(b)の平均の差に対して有意差検定を実行する
stats.ttest_ind(mens_mail, no_mail)

Ttest_indResult(statistic=5.300090294465472, pvalue=1.163200872605869e-07)

In [8]:
# (7) セレクションバイアスのあるデータの作成
## seedを固定する
random.seed(0)

In [9]:
## 条件に反応するサンプルの量を半分にする
obs_rate_c = 0.5
obs_rate_t = 0.5

In [10]:
## バイアスのあるデータの作成
male_df['obs_rate_c'] = np.where((male_df['history'] > 300) |  (male_df['recency'] < 6) | (male_df['channel']== 'Multichannel'),
                                                             obs_rate_c, 1)
male_df['obs_rate_t'] = np.where((male_df['history'] > 300) |  (male_df['recency'] < 6) | (male_df['channel']== 'Multichannel'),
                                                             1, obs_rate_t)
male_df['random_number'] = [random.random() for _ in range(len(male_df))]
biased_data = male_df.query('(treatment == 0 and random_number < obs_rate_c) or (treatment == 1 and random_number < obs_rate_t)').reset_index(drop=True)

In [11]:
# (8) セレクションバイアスのあるデータで平均を比較
## group_byとsummairseを使って集計(Biased)
biased_data.groupby('treatment')[['conversion', 'spend']].agg(['mean', 'count'])

conversion            spend       
                mean  count      mean  count
treatment                                   
0           0.005413  14779  0.616096  14779
1           0.013368  17131  1.553958  17131

In [12]:
# (9) Rの関数であるt.testを使ってt検定を行う(Biased)
## (a)男性向けメールが配信されたグループの購買データを得る
mens_mail_biased = biased_data.query('treatment == 1')['spend']
## (b)メールが配信されなかったグループの購買データを得る
no_mail_biased = biased_data.query('treatment == 0')['spend']

In [13]:
## (a)(b)の平均の差に対して有意差検定を実行
stats.ttest_ind(mens_mail_biased, no_mail_biased)

Ttest_indResult(statistic=5.30559632407586, pvalue=1.1305338001766832e-07)